# Inputting game stats
-note the data is 2015

In [171]:
import numpy as np

class GameStats(object):          
    
    def __init__(self, homeTeamNameIndex, homeTeamScoreIndex, homeTeamStatsIndex, visitorTeamNameIndex, visitorTeamScoreIndex, visitorTeamStatsIndex):
        #parse the text file
        self.statsFile = open("baseball2015.txt", "r")
        self.topArray = []
        self.sideArray = []  
        self.sc = np.zeros((30,30,30), np.int32) 
        self.sc[:,:,:] = -1  
        self.am = np.zeros((30,30), np.float32)
        self.gameList = []
        
        seenTeams = []
        for line in self.statsFile:
            token = line.split(',')  #tokenize the string
            tokenIndex = [homeTeamNameIndex, homeTeamScoreIndex, visitorTeamNameIndex, visitorTeamScoreIndex] + [i for i in homeTeamStatsIndex] + [i for i in visitorTeamStatsIndex]
            attributes = dict()
            
            for i in range(len(token)):
                if(i in tokenIndex):
                    attributes[i] = self.removeQuotes(token[i])
                        
            self.addScore(attributes[homeTeamNameIndex], attributes[visitorTeamNameIndex], attributes[homeTeamScoreIndex], attributes[visitorTeamScoreIndex])                
            self.addGame(attributes[homeTeamNameIndex], attributes[homeTeamScoreIndex], [attributes[i] for i in homeTeamStatsIndex], attributes[visitorTeamNameIndex], attributes[visitorTeamScoreIndex], [attributes[i] for i in visitorTeamStatsIndex])
            
            if(attributes[homeTeamNameIndex] not in seenTeams):
                seenTeams.append(attributes[homeTeamNameIndex])
            
        self.buildAvgMatrix()
        self.statsFile.close()
        #self.gameList = [bin(2**i)[2:].zfill(len(seenTeams)) if x == seenTeams[i] else x for i in range(len(seenTeams)) for x in self.gameList]
        # take the teams and convert the indexes of the order that they appeared in into the hot one format by rasiing
        # 2 to the power of them and then converting them into binary.
        seenTeamsDict = {k: v for v, k in enumerate(seenTeams)}
        temp = []
        for x in self.gameList:
            tempi = []
            for z in x:
                if(z in seenTeams):
                    tempi.append(bin(2**seenTeamsDict[z])[2:].zfill(len(seenTeamsDict)))
                else:
                    tempi.append(z)
            temp.append(tempi)
        self.gameList = temp
        
    def removeQuotes(self, string):
        if (string.startswith('"') and string.endswith('"')) or (string.startswith("'") and string.endswith("'")):
            return string[1:-1]
        return string  
    
    def addGame(self, team1, score1, stats1, team2, score2, stats2):
        self.gameList.append([team1, score1, stats1, team2, score2, stats2])
        
    #give it two teams, the scores, and it will add it to the matrix
    def addScore(self, team1, team2, score1, score2):
        '''
        for a team in top array, the index in the array corrisponds to the matrix column there located in
        for a team in side array, the index in the array corrisponds to the matrix row there located in
        '''
        #team 1 score entry
        try:
            row = self.sideArray.index(team2)    

        except:
            self.sideArray.append(team2)
            row = self.sideArray.index(team2)    

        try:
            col = self.topArray.index(team1)
        except:
            self.topArray.append(team1)
            col = self.topArray.index(team1)
        temp = self.sc[row, col]
        counter = 0
        for e in temp:
            if (e == -1):
                temp[counter] = score1
                break
            counter += 1
        self.sc[row, col] = temp
        
        #team 2 score entry
        try:
            row = self.sideArray.index(team1)    
        except:
            self.sideArray.append(team1)
            row = self.sideArray.index(team1)    
            
        try:
            col = self.topArray.index(team2)
        except:
            self.topArray.append(team2)
            col = self.topArray.index(team2)
        temp = self.sc[row, col]
        counter = 0
        for e in temp:
            if (e == -1):
                temp[counter] = score2
                break
            counter += 1
        self.sc[row, col] = temp
    
    #returns the score(s) for match up
    def getScore(self, team1, team2, gameSelect = None):
        print(team1, team2)
        try:
            score1 = self.sc[self.sideArray.index(team2), self.topArray.index(team1)]
            score2 = self.sc[self.sideArray.index(team1), self.topArray.index(team2)]
            if (gameSelect == None):
                print(team1, score1)
                print(team2, score2)
            else:
                print(team1, score1[gameSelect])
                print(team2, score2[gameSelect])
        except:
            print('Invalid input of teams')
    
    def getGameList(self):
        return self.gameList
    
    #constructs a matrix of the avg score in a matchup
    def buildAvgMatrix(self): 
        for col in range(len(self.sc[:,0])):   #depth
            for row in range(len(self.sc[0, :])):  #width
                tempScore = self.sc[row, col]
                avgScore = 0.0
                count = 0.0
                for j in tempScore:
                    if (j != -1):
                        avgScore += j
                        count += 1
                    else:
                        break
                try:
                    avgScore = avgScore / count
                except:
                    avgScore = -1
                self.am[row, col] = avgScore
    
    #get the value of the avg score for a match up
    def getAvgScore(self, team1, team2):
        try:
            score1 = self.am[self.sideArray.index(team2), self.topArray.index(team1)]
            score2 = self.am[self.sideArray.index(team1), self.topArray.index(team2)]
            print(team1, score1)
            print(team2, score2)        
        except:
            print('Invalid input of teams')

In [179]:
import tensorflow as tf
import tflearn

tf.reset_default_graph()

# Parse the game files and grab the stats we want
# the indexes are put in the order: homeTeamName, homeTeamScore, homeTeamStats, awayName, awayScore, awayStats
stats = GameStats(3, 10, [5, 51, 55], 6, 9, [8, 23, 27])
gameList = stats.getGameList() # get the list of games
avgStats = dict()

#BINARY STYLE NUMBERS ARE IN THE GAME STATS MINI ARRAYS!!!
for row in gameList:
    print(row[2], row[5])

(['1', '3', '0'], ['1', '3', '0'])
(['1', '1', '0'], ['1', '0', '0'])
(['1', '0', '0'], ['1', '0', '0'])
(['1', '3', '1'], ['1', '1', '0'])
(['1', '0', '0'], ['1', '0', '1'])
(['1', '3', '0'], ['1', '0', '0'])
(['1', '3', '0'], ['1', '0', '0'])
(['1', '2', '0'], ['1', '2', '0'])
(['1', '1', '0'], ['1', '3', '0'])
(['1', '0', '0'], ['1', '1', '0'])
(['1', '4', '0'], ['1', '4', '0'])
(['1', '1', '0'], ['1', '1', '1'])
(['1', '0', '0'], ['1', '6', '1'])
(['1', '1', '0'], ['1', '0', '0'])
(['1', '0', '0'], ['1', '0', '0'])
(['2', '0', '0'], ['2', '1', '0'])
(['2', '0', '0'], ['2', '2', '0'])
(['2', '0', '0'], ['2', '1', '0'])
(['2', '1', '0'], ['2', '4', '0'])
(['2', '4', '0'], ['2', '1', '1'])
(['2', '0', '0'], ['2', '4', '2'])
(['2', '3', '0'], ['2', '6', '0'])
(['2', '5', '0'], ['2', '1', '0'])
(['2', '1', '0'], ['2', '0', '1'])
(['2', '1', '0'], ['2', '3', '0'])
(['2', '2', '0'], ['2', '1', '0'])
(['3', '3', '0'], ['3', '1', '1'])
(['3', '2', '0'], ['3', '2', '1'])
(['3', '1', '0'], ['

In [172]:
#getting rid of the strings
for row in gameList:
    for x in xrange(len(row)):
        #convert scores strings to float
        if (x == 1 or x == 4): row[x] = float(row[x])
        #convert arrays to floats
        if (x == 2 or x == 5): row[x] = map(float, row[x])

#adding up all the teams stats
for row in gameList:
    #visitors
    print("visitor", row[0])
    if avgStats.has_key(row[0]):
        print(avgStats[row[0]])
        avgStats[row[0]][0] = 0   #set the proir games to 0, so because there set to right amount next line
        avgStats[row[0]] = np.sum([avgStats[row[0]], row[2]], axis=0)  #total stats + indivudal game stats
    else:
        avgStats[row[0]] = row[2]
    print(row[2])
    print(avgStats[row[0]])
    print('-------------')
    
    #home
    print("home", row[3])
    if avgStats.has_key(row[3]):
        
        print(avgStats[row[3]])
        avgStats[row[3]][0] = 0
        avgStats[row[3]] = np.sum([avgStats[row[3]], row[5]], axis=0)
    else:
        avgStats[row[3]] = row[5]
    print(row[5])
    print(avgStats[row[3]])
    print('-------------')

#make the average..
    
    

('visitor', '000000000000000000000000000001')
[1.0, 3.0, 0.0]
[1.0, 3.0, 0.0]
-------------
('home', '000000000100000000000000000000')
[1.0, 3.0, 0.0]
[1.0, 3.0, 0.0]
-------------
('visitor', '000000000000000000000000000010')
[1.0, 1.0, 0.0]
[1.0, 1.0, 0.0]
-------------
('home', '000000000000010000000000000000')
[1.0, 0.0, 0.0]
[1.0, 0.0, 0.0]
-------------
('visitor', '000000000000000000000000000100')
[1.0, 0.0, 0.0]
[1.0, 0.0, 0.0]
-------------
('home', '000000000001000000000000000000')
[1.0, 0.0, 0.0]
[1.0, 0.0, 0.0]
-------------
('visitor', '000000000000000000000000001000')
[1.0, 3.0, 1.0]
[1.0, 3.0, 1.0]
-------------
('home', '000000000000001000000000000000')
[1.0, 1.0, 0.0]
[1.0, 1.0, 0.0]
-------------
('visitor', '000000000000000000000000010000')
[1.0, 0.0, 0.0]
[1.0, 0.0, 0.0]
-------------
('home', '000000100000000000000000000000')
[1.0, 0.0, 1.0]
[1.0, 0.0, 1.0]
-------------
('visitor', '000000000000000000000000100000')
[1.0, 3.0, 0.0]
[1.0, 3.0, 0.0]
-------------
('h

In [40]:
# the input layer needs to have the same dimensions as our input (in this case the teams)
net = tflearn.input_data(shape=[None, len(gameList[0][0])], name='input')
# next we have the hidden layer it is the feature matrix the size is arbitrary.
# I chose 15 becasue that is what the matrix factorization feature size is set to.
net = tflearn.fully_connected(net, 15)
# The output layer
net = tflearn.fully_connected(net, len(gameList[0][2]))
net = tflearn.regression(net, name='target')

# Note(daniel): right now we don't care what teams played eachother (we may in the future) therefore all of the teams are put
# into one array.

# take only the team names and split them into individual parts (ie 0100 becomes [0], [1], [0], [0])
NNInput = [list(i[x]) for i in gameList for x in range(len(i)) if x == 0 or x == 3]
NNInput = np.array(NNInput)

# take only the stats for each team and put them into an array
NNOutput = [i[x] for i in gameList for x in range(len(i)) if x == 2 or x == 5]
NNOutput = np.array(NNOutput)

# Define model
model = tflearn.DNN(net)
# Start training (apply gradient descent algorithm)
model.fit(NNInput, NNOutput, validation_set=0.1, n_epoch=20, show_metric=True)

Training Step: 1379  | total loss: 0.62207 | time: 0.338s
| Adam | epoch: 020 | loss: 0.62207 - acc: 0.9722 -- iter: 4352/4370
Training Step: 1380  | total loss: 0.62612 | time: 1.348s
| Adam | epoch: 020 | loss: 0.62612 - acc: 0.9718 | val_loss: 19333.10720 - val_acc: 0.9630 -- iter: 4370/4370
--
